In [61]:
import time
import numpy as np
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [62]:
from FADAnet.FADAloader import *
from FADAnet.FADAmodule import *

In [63]:
n_epoch = 100
batch_size = 1024
lr = 0.001
n_support = 1

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
np.random.seed(717)
torch.manual_seed(717)
torch.cuda.manual_seed(717)

In [64]:
train_dataloader=mnist_dataloader(batch_size=batch_size,train=True)
test_dataloader=mnist_dataloader(batch_size=batch_size,train=False)

classifier=Classifier()
encoder=Encoder()

classifier.to(device)
encoder.to(device)
loss_fn=torch.nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(list(encoder.parameters())+list(classifier.parameters()))

In [ ]:
# for epoch in tqdm(range(n_epoch)):
for epoch in range(n_epoch):
    for data,labels in train_dataloader:
        data=data.to(device)
        labels=labels.to(device)

        optimizer.zero_grad()

        y_pred=classifier(encoder(data))

        loss=loss_fn(y_pred,labels)
        loss.backward()

        optimizer.step()
    if epoch%10 == 9:
        acc=0
        for data,labels in test_dataloader:
            data=data.to(device)
            labels=labels.to(device)
            y_test_pred=classifier(encoder(data))
            acc+=(torch.max(y_test_pred,1)[1]==labels).float().mean().item()

        accuracy=round(acc / float(len(test_dataloader)), 3)
        print("step1----Epoch %d/%d  accuracy: %.3f "%(epoch+1,n_epoch,accuracy))

In [93]:
mapset = []
labelset = []
for data, labels in train_dataloader:
    data=data.to(device)
    fmap = encoder(data).cpu().detach().numpy()
    labels=labels.to(device).cpu().detach().numpy()
    mapset.append(fmap)
    labelset.append(labels)

In [108]:
smap = np.vstack(mapset[:-1])
slabel = np.hstack(labelset[:-1])

In [164]:
means = []
dmeans = []

for num in range(10):
    subset1 = smap[slabel == num]
    means1 = np.mean(subset1, axis=0)
    tmp = subset1 - means1
    dists1 = np.linalg.norm(tmp, axis=1)
    means.append(means1)
    dmeans.append(np.mean(dists1))

print(dmeans)
for i in range(10):
    for j in range(i+1):
        print(np.linalg.norm(means[i] - means[j]), end=',')
    print()


[20.945534, 12.921637, 32.431023, 24.667782, 22.536781, 24.352346, 20.08793, 25.857838, 24.888908, 23.244738]
0.0,
68.765076,0.0,
78.48123,75.871666,0.0,
101.824554,82.75783,84.97293,0.0,
86.03646,62.545166,101.36127,113.2429,0.0,
81.95166,78.888626,113.66938,72.81477,94.375854,0.0,
64.88343,74.306984,98.519745,108.76796,78.491684,69.95229,0.0,
85.633644,69.39298,78.76395,73.16075,93.74609,96.8604,119.15769,0.0,
77.69174,67.552124,80.01183,72.787476,83.872406,68.49606,67.304214,90.76718,0.0,
80.36195,74.221535,98.247536,79.31981,65.0958,74.38497,94.88097,66.936134,69.27747,0.0,
